In [ ]:
pwd

In [ ]:
import sys
sys.path.append("/p/project/raise-ctp2/cern/particleflow/mlpf")

In [ ]:
# mlpf pipeline imports
import os
import yaml
import json
import datetime
import glob
import random
import platform
import numpy as np
from pathlib import Path
import click
from tqdm import tqdm
import shutil
from functools import partial
import shlex
import subprocess

import tensorflow as tf
from tensorflow.keras import mixed_precision
import tensorflow_addons as tfa
import keras_tuner as kt

In [ ]:
import ray
from ray import tune
from ray.tune.integration.keras import TuneReportCheckpointCallback
from ray.tune.integration.tensorflow import DistributedTrainableCreator
from ray.tune.logger import TBXLoggerCallback
from ray.tune import Analysis, ExperimentAnalysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from matplotlib import rc_file
rc_file("my_matplotlib_rcparams.txt")

In [ ]:
##### FZJ #####
ray_results_folder = "/p/project/raise-ctp2/cern/ray_results/"  # Main folder containing all ray experiments
###############


##### Flatiron #####
ray_results_folder = "/mnt/ceph/users/ewulff/ray_results/"  # Main folder containing all ray experiments
exp_dir = ray_results_folder + "clic_gnn_search_asha_hyperopt_n500"
####################

In [ ]:
ll $ray_results_folder

In [ ]:
# Which experiments to analyze
exp_dir_list = [# ray_results_folder + "cmsgen_asha_epochs50",
                # ray_results_folder + "cmsgen_asha_epochs100",
                # ray_results_folder + "cmsgen_asha_scikit_epochs50_samples400_OneGPUPerTrial",
                # ray_results_folder + "cmsgen_asha_scikit_epochs50_samples200_FourGPUPerTrial",
                # ray_results_folder + "clic_transformer_search_asha_n500",
                ray_results_folder + "clic_transformer_search_asha_hyperopt_n500",
               ]

##### Flatiron
exp_dir_list = [ray_results_folder + "clic_gnn_scan",
                ray_results_folder + "clic_gnn_scan_epochs3500",
                ray_results_folder + "clic_gnn_search_asha_hyperopt_n500",
               ]

In [ ]:
default_mode = "min"
default_metric = "val_loss"
expanalysis_list = []
for exp_dir in exp_dir_list:
    expanalysis_list.append(ExperimentAnalysis(exp_dir, default_metric=default_metric, default_mode=default_mode))

In [ ]:
def robust_get_df(analysis):
    try:
        result_df = analysis.dataframe()
    except IndexError:
        result_df = analysis.results_df.dropna(axis=0, how="all")
    return result_df

In [ ]:
print("Time from start to timestamp:")
for expanalysis in expanalysis_list:
    res_df = robust_get_df(expanalysis)
    exp_dir = Path(res_df['logdir'][0]).parent
    stats = expanalysis.stats() 
    print("{:.2f}".format((stats["timestamp"] - stats["start_time"]) / 60 / 60), "h", exp_dir)

In [ ]:
res_df.keys()

In [ ]:
def summarize_best_results(expanalysis_list):
    summ = []
    for expanalysis in expanalysis_list:
        res_df = robust_get_df(expanalysis)
        exp_dir = Path(res_df['logdir'][0]).parent

        dd = expanalysis.best_result
        val_reg_loss = sum([dd["val_{}_loss".format(l)] for l in ["energy", "pt", "eta", "sin_phi", "cos_phi", "charge"]])
        reg_loss = sum([dd["{}_loss".format(l)] for l in ["energy", "pt", "eta", "sin_phi", "cos_phi", "charge"]])

        conf = expanalysis.best_result.pop("config")

        expanalysis.best_result.update({"val_reg_loss": val_reg_loss, "reg_loss": reg_loss})
        expanalysis.best_result.update({x: conf[x] for x in conf})

        summ.append(pd.DataFrame(expanalysis.best_result, index=[exp_dir]))
        expanalysis.best_result.update({"config": conf})
    return pd.concat(summ)
    

In [ ]:
summary = summarize_best_results(expanalysis_list)

## Don't forget to consider which datases the models were tuned on and tested. That affects all evaluation metrics.

In [ ]:
interesting = ["loss", "val_loss",
               "cls_loss", "val_cls_loss",
               "reg_loss", "val_reg_loss",
               # 'cls_acc_unweighted', 'val_cls_acc_weighted',
               # 'cls_acc_unweighted', 'val_cls_acc_unweighted',
              ]
interesting += [x for x in expanalysis.best_result["config"].keys()]
summary.sort_values(default_metric)
summary[interesting].transpose()